In [1]:
import pandas as pd
import numpy as np
np.random.seed(13)
from datetime import date
import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('./data/listings-5.csv')
pd.set_option('display.max_columns', len(df.columns)) 
pd.set_option('display.max_rows', 100)

/var/folders/yw/mzjjwqwd12g6lm2qkdn097dw0000gn/T/ipykernel_10335/296277742.py:1: DtypeWarning: Columns (68) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/listings-5.csv')


### Data Cleaning

In [3]:
df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2539,https://www.airbnb.com/rooms/2539,20220907064715,2022-09-07,city scrape,Clean & quiet apt home by the park,Renovated apt home in elevator building.<br />...,Close to Prospect Park and Historic Ditmas Park,https://a0.muscache.com/pictures/3949d073-a02e...,2787,https://www.airbnb.com/users/show/2787,John,2008-09-07,"New York, NY",Educated professional living in Brooklyn. I l...,within an hour,100%,90%,f,https://a0.muscache.com/im/pictures/user/86745...,https://a0.muscache.com/im/pictures/user/86745...,Gravesend,9.0,12.0,"['email', 'phone']",t,t,"Brooklyn , New York, United States",Kensington,Brooklyn,40.64529,-73.97238,Private room in rental unit,Private room,2,NaN,1 shared bath,1.0,1.0,"[""First aid kit"", ""Microwave"", ""Stove"", ""Coffe...",$299.00,30,730,30.0,30.0,730.0,730.0,30.0,730.0,NaN,t,21,51,81,356,2022-09-07,9,0,0,2015-12-04,2018-10-19,4.89,4.88,5.00,5.00,5.00,4.75,4.88,NaN,f,9,1,6,2,0.11
1,2595,https://www.airbnb.com/rooms/2595,20220907064715,2022-09-07,city scrape,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...",Centrally located in the heart of Manhattan ju...,https://a0.muscache.com/pictures/f0813a11-40b2...,2845,https://www.airbnb.com/users/show/2845,Jennifer,2008-09-09,"New York, NY",A New Yorker since (Phone number hidden by Air...,within a day,75%,23%,f,https://a0.muscache.com/im/pictures/user/50fc5...,https://a0.muscache.com/im/pictures/user/50fc5...,Midtown,6.0,9.0,"['email', 'phone', 'work_email']",t,t,"New York, United States",Midtown,Manhattan,40.75356,-73.98559,Entire rental unit,Entire home/apt,1,NaN,1 bath,NaN,1.0,"[""Stove"", ""Coffee maker"", ""Long term stays all...",$175.00,30,1125,30.0,30.0,1125.0,1125.0,30.0,1125.0,NaN,t,0,0,5,280,2022-09-07,49,1,0,2009-11-21,2022-06-21,4.68,4.73,4.63,4.77,4.80,4.81,4.40,NaN,f,3,3,0,0,0.31
2,5121,https://www.airbnb.com/rooms/5121,20220907064715,2022-09-07,city scrape,BlissArtsSpace!,One room available for rent in a 2 bedroom apt...,NaN,https://a0.muscache.com/pictures/2090980c-b68e...,7356,https://www.airbnb.com/users/show/7356,Garon,2009-02-03,"New York, NY","I am an artist(painter, filmmaker) and curato...",within an hour,100%,100%,t,https://a0.muscache.com/im/pictures/user/72a61...,https://a0.muscache.com/im/pictures/user/72a61...,Bedford-Stuyvesant,2.0,2.0,"['email', 'phone']",t,t,NaN,Bedford-Stuyvesant,Brooklyn,40.68535,-73.95512,Private room in rental unit,Private room,2,NaN,NaN,1.0,1.0,"[""Heating"", ""Kitchen"", ""Air conditioning"", ""Wi...",$60.00,30,730,30.0,30.0,730.0,730.0,30.0,730.0,NaN,t,5,30,60,335,2022-09-07,50,0,0,2009-05-28,2019-12-02,4.52,4.22,4.09,4.91,4.91,4.47,4.52,NaN,f,2,0,2,0,0.31
3,45910,https://www.airb

I began with making sure the columns are more pythonic by lowercasing the columns

In [4]:
df.columns.str.lower()

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

I began with looking at the null values and see which columns had missing values which are indicated by the isnull. 


In [5]:
df.isnull().sum()

id                                                  0
listing_url                                         0
scrape_id                                           0
last_scraped                                        0
source                                              0
name                                               13
description                                       845
neighborhood_overview                           16415
picture_url                                         0
host_id                                             0
host_url                                            0
host_name                                          50
host_since                                         50
host_location                                    6883
host_about                                      17372
host_response_time                              13548
host_response_rate                              13548
host_acceptance_rate                            11861
host_is_superhost           

When looking at the dtypes for each of the columns, there were some columns that weren't in the right format. The columns I saw that weren't in the right format were host_since, host_response_time, host_response_rate, host_acceptance_rate, host_is_superhost, host_verfication, host_has_profile_pic, host_identity_verified, price, first_review, and last_review. 

In [6]:
df.dtypes

id                                                int64
listing_url                                      object
scrape_id                                         int64
last_scraped                                     object
source                                           object
name                                             object
description                                      object
neighborhood_overview                            object
picture_url                                      object
host_id                                           int64
host_url                                         object
host_name                                        object
host_since                                       object
host_location                                    object
host_about                                       object
host_response_time                               object
host_response_rate                               object
host_acceptance_rate                            

I began by first dropping the url columns because they won't be helpful when it predicting models because they all are just links. 

In [7]:
url_cols = [cols for cols in df.columns if cols.endswith('url')]
df.drop(columns = url_cols, inplace = True)

We are going to be predicting prices in New York, as a result, scraping information won't be useful when it comes to predicting the models.so those will be dropped. 

In [8]:
df.drop(columns = ['scrape_id', 'last_scraped', 'source', 'calendar_last_scraped'], inplace = True) 

Name, description, and neighborhood_overview all provide a description of the airbnb thats going to be rented, but due to the model not including NLP, these columns won't be helpful.

In [9]:
df.drop(columns = ['name', 'description', 'neighborhood_overview'], inplace = True)

The columns such as host_name, host_location, host_about, host_response_time, host_verifications, neighbourhood, host_neighbourhood, neighbourhood_group_cleansed, and host_since provide descriptions of the person living in a airbnb location and so since this information does not provide any predictive importantance to determine airbnb prices. These columns were removed as a result. 

In [10]:
df.drop(columns = ['host_name', 'host_location', 'host_about', 'host_response_time', 'host_verifications', 'neighbourhood', 
                   'host_neighbourhood', 'neighbourhood_group_cleansed', 'host_since'], inplace = True)

The bathroom column is filled with all null values, so that column will be dropped as well as calendar updated and license was all null values.

In [11]:
df.drop(columns = ['bathrooms', 'calendar_updated', 'license'], inplace = True)

The different columns that show minimum_minimum_nights is the same as minimum_nights, so that can be dropped. When it comes to maximum_minimum_nights, it seems that they are still as the minimum_nights so that will be dropped. The minimum_maximum_nights has the same values as the minimum_maximum_nights, so that will be dropped. minimum_minimum_nights and maximum_maximum_nights have the same value as maximum_nights.

In [12]:
df.drop(columns = ['minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 
                   'minimum_nights_avg_ntm','maximum_nights_avg_ntm'], inplace = True)

The last_review and first_review will be dropped because when looking at the columns they only inform dates on the visit which provides no detail unless they provided a rating system or we are looking at the time series on the first_review and last_review. 

In [13]:
df.drop(columns = ['last_review', 'first_review'], inplace = True)

The host_id will be dropped because we will be following the id column to represent each of the different airbnb.

In [14]:
df.drop(columns = 'host_id', inplace = True)

The file called listings will have all the counts that will be used for my data analysis and so the columns such as calculated_host_listings_count, calculated_host_listings_count_entire_homes, calculated_host_listings_count_private_rooms, calculated_host_listings_count_shared_rooms, and host_total_listings_count will be removed as they provide information that is present in the listings file.

In [15]:
df.drop(columns = ['calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms',
                   'calculated_host_listings_count_shared_rooms', 'host_total_listings_count'], inplace = True)

### Host Response rate

The first part that needs to be taken care of is that host_response_rate in the dtypes is found to be an object instead of a float or int, so it will first be changed into a float. It would be helpful to look at the mean for the host response rate. The mean response rate was 0.93, which is a high response rate

In [16]:
df['host_response_rate'] = df['host_response_rate'].str.rstrip("%").astype(float)/100
print(f' Mean AirBnb host response rate : {df.host_response_rate.mean()}')

#https://levelup.gitconnected.com/convert-percentage-string-to-numeric-and-vice-versa-in-pandas-18a3d66e2853

 Mean AirBnb host response rate : 0.9368028709224128


We will next look at the number of null values. There are over 13,548 columns in which the values are null.

In [17]:
df['host_response_rate'].isnull().sum()

13548

Next, I will look at the value counts for host response rate and it seems to be spread out in numbers. As presented, 1.00 has the highest count compared to all the other rates. 

In [18]:
df['host_response_rate'].value_counts()

1.00    17842
0.90      968
0.99      897
0.98      566
0.96      550
0.94      466
0.80      464
0.00      356
0.92      339
0.97      337
0.83      246
0.50      230
0.91      215
0.70      213
0.93      185
0.67      184
0.75      179
0.95      178
0.89      149
0.88      148
0.60      145
0.78      120
0.43      119
0.40      110
0.86      107
0.87       98
0.71       70
0.81       64
0.82       55
0.73       55
0.59       54
0.33       45
0.85       41
0.76       41
0.79       38
0.77       33
0.56       32
0.25       29
0.52       28
0.63       26
0.69       26
0.57       22
0.64       21
0.84       19
0.10       18
0.74       17
0.66       17
0.18       16
0.30       16
0.20       16
0.17        9
0.55        8
0.42        8
0.65        8
0.58        7
0.72        7
0.38        6
0.29        5
0.46        5
0.62        5
0.36        5
0.54        5
0.13        5
0.48        5
0.05        5
0.11        4
0.68        3
0.44        3
0.35        3
0.14        2
0.47        2
0.53  

### Host Acceptance rate

The first part that needs to be taken care of is that host_acceptance_rate in the dtypes is found to be an object instead of a float or int, so it will first be changed into a float. It would be helpful to look at the mean for the host acceptance rate. The mean acceptance rate was 0.823, which is a high acceptance rate.

In [19]:
df['host_acceptance_rate'] = df['host_acceptance_rate'].str.rstrip("%").astype(float)/100
print(f' Mean AirBnb host acceptance rate : {df.host_acceptance_rate.mean()}')

 Mean AirBnb host acceptance rate : 0.8234950035688555


The column has over 11000 null values, I dont want to drop them, so Ill be doing an iterative imputer to fill in the null values

In [20]:
df['host_acceptance_rate'].isnull().sum()

11861

Next, I will look at the value counts for host acceptance rate and it seems to be spread out in numbers. As presented, 1.00 has the highest count compared to all the other rates. 

In [21]:
df['host_acceptance_rate'].value_counts()

1.00    8802
0.99    1521
0.97    1517
0.98    1215
0.00    1086
0.96     823
0.92     780
0.94     597
0.88     577
0.86     576
0.90     549
0.67     517
0.95     509
0.50     491
0.93     467
0.89     444
0.91     360
0.75     353
0.80     316
0.05     289
0.83     287
0.78     267
0.33     232
0.87     221
0.81     221
0.79     221
0.60     220
0.85     210
0.82     209
0.71     198
0.84     185
0.70     176
0.77     166
0.73     152
0.62     142
0.56     135
0.63     129
0.45     128
0.76     123
0.69     121
0.65     116
0.25     107
0.54     101
0.64      92
0.44      92
0.57      90
0.46      89
0.72      86
0.40      86
0.29      80
0.19      79
0.59      76
0.55      75
0.43      72
0.61      71
0.38      68
0.68      68
0.58      58
0.74      57
0.20      55
0.53      48
0.42      45
0.41      42
0.39      41
0.47      39
0.66      37
0.14      36
0.17      33
0.36      33
0.30      32
0.52      31
0.28      29
0.48      29
0.11      26
0.49      25
0.09      25
0.31      25

#### Host_is_superhost

Host is superhost column was true and false as a result, the first part was to conver it into a float in which false is a 0 and true is 1. The column was also an object, so I converted it into float

In [22]:
df['host_is_superhost'] = df['host_is_superhost'].map({'f':0, 't':1}).astype(float)

First, we look at the value counts for the host_is_superhost. There are a larger number of no superhost compared to superhost in the New York City area. 

In [23]:
df['host_is_superhost'].value_counts(normalize = True)

0.0    0.794966
1.0    0.205034
Name: host_is_superhost, dtype: float64

Then we will look at the null values in this column

In [24]:
df['host_is_superhost'].isnull().sum()

29

I will look into each of the 29 rows to see if there is a significant effect if I dropped the null values. From the Data, it seems that by dropping the host_is_superhost it will also get rid of the null values for minimum_nights_avg_ntm and maximum_nights_abg_ntm.

In [25]:
df[df['host_is_superhost'].isna()]

,id,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month
20784,40560656,1.00,1.00,NaN,2.0,t,t,Williamsburg,40.720960,-73.958600,Room in boutique hotel,Hotel room,2,NaN,NaN,NaN,"[""First aid kit"", ""Toiletries"", ""Coffee maker""...",$0.00,1,365,t,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN
21467,41740615,1.00,1.00,NaN,5.0,t,t,Midtown,40.744590,-73.985740,Room in boutique hotel,Hotel room,4,NaN,NaN,NaN,"[""First aid kit"", ""Appelles toiletries"", ""40\""...",$0.00,1,28,t,0,0,0,0,44,13,1,4.36,4.45,4.64,4.39,4.36,4.66,4.32,f,1.27
21469,41740622,1.00,0.93,NaN,6.0,t,t,Upper East Side,40.764420,-73.963030,Room in hotel,Hotel room,6,NaN,NaN,NaN,"[""First aid kit"", ""[ comfort zone ] toiletries...",$0.00,1,28,t,0,0,0,0,4,3,0,4.50,3.75,4.50,4.50,4.50,4.50,4.00,f,0.15
21581,42065543,1.00,1.00,NaN,5.0,t,t,Midtown,40.744440,-73.989200,Room in boutique hotel,Hotel room,4,NaN,NaN,NaN,"[""First aid kit"", ""Housekeeping"", ""Coffee make...",$0.00,1,365,t,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN
21589,42065545,NaN,NaN,NaN,4.0,t,t,Midtown,40.759170,-73.969260,Room in boutique hotel,Hotel room,4,NaN,NaN,NaN,"[""First aid kit"", ""Toiletries"", ""Coffee maker""...",$0.00,30,365,t,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN
21590,42065547,1.00,0.92,NaN,1.0,t,t,Hell's Kitchen,40.764040,-73.994780,Room in hotel,Hotel room,4,NaN,NaN,NaN,"[""Hair dryer"", ""Luggage dropoff allowed"", ""Fir...",$0.00,30,365,t,0,0,0,0,36,21,0,4.17,4.44,4.19,4.61,4.03,4.75,4.36,f,1.51
21592,42065555,NaN,NaN,NaN,3.0,t,t,Williamsburg,40.715230,-73.959080,Room in hotel,Hotel room,2,NaN,NaN,NaN,"[""First aid kit"", ""Toiletries"", ""Long term sta...",$0.00,30,365,f,0,0,0,0,1,0,0,5.00,5.00,5.00,5.00,5.00,5.00,5.00,f,0.03
21593,42065562,1.00,1.00,NaN,6.0,t,t,Financial District,40.709580,-74.008740,Room in hotel,Hotel room,4,NaN,NaN,NaN,"[""CO Bigelow toiletries"", ""First aid kit"", ""Ba...",$0.00,1,28,t,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN
21602,42065563,NaN,NaN,NaN,2.0,t,t,Mott Haven,40.815130,-73.916020,Room in hotel,Hotel room,4,NaN,NaN,NaN,"[""First aid kit"", ""Toiletries"", ""Complimentary...",$0.00,30,365,t,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN
21657,42279171,1.00,0.98,NaN,7.0,t,t,Chinatown,40.716100,-73.995180,Room in boutique hotel,Hotel room,6,NaN,NaN,NaN,"[""Luggage dropoff allowed"", ""First aid kit"", ""...",$0.00,1,365,t,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN


I dropped the null values for the host_is_superhost

In [26]:
df.dropna(subset = 'host_is_superhost', inplace = True)

### Host_listing_count

Begin by looking at the value count for the column. The values are spread out from 1 to 276. 1 seems to have the highest number when its comes to count at 19651. 

In [27]:
df['host_listings_count'].value_counts()

1.0      19651
2.0       5793
3.0       2953
4.0       1737
5.0       1031
         ...  
38.0         1
39.0         1
111.0        1
90.0         1
276.0        1
Name: host_listings_count, Length: 109, dtype: int64

I looked into the null values in this column and its interesting that a lot of information about the hist is null for other columns such as host_since, host_response_rate, host_acceptance_rate, and etc. The 50 null values may be indication that the identity of the host is unknown, so these will be dropped. 

In [28]:
df['host_listings_count'].isnull().sum()

50

I looked into the null values to see if any important columns were going to be dropped. As you can see the 50 columns have a large number of null values in other columns as well.

In [29]:
df[df['host_listings_count'].isna()]

,id,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month
3013,4446862,NaN,NaN,0.0,NaN,NaN,NaN,Crown Heights,40.67605,-73.95997,Private room in rental unit,Private room,2,1 bath,1.0,1.0,"[""Essentials"", ""Heating"", ""Smoke alarm"", ""TV"",...",$50.00,30,1125,t,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN
3315,4763327,NaN,NaN,0.0,NaN,NaN,NaN,Greenpoint,40.72119,-73.95279,Entire rental unit,Entire home/apt,4,1 bath,1.0,2.0,"[""Essentials"", ""Pool"", ""Cable TV"", ""Elevator"",...",$195.00,30,1125,f,0,0,0,0,1,0,0,4.0,4.00,5.00,5.00,4.00,5.0,4.00,f,0.01
4166,6360224,NaN,NaN,0.0,NaN,NaN,NaN,Bushwick,40.69977,-73.92912,Private room in rental unit,Private room,1,0 baths,1.0,1.0,"[""First aid kit"", ""Essentials"", ""Heating"", ""Sm...",$37.00,30,1125,f,0,0,0,0,1,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,f,0.01
4689,6992973,NaN,NaN,0.0,NaN,NaN,NaN,Williamsburg,40.71944,-73.95645,Entire rental unit,Entire home/apt,4,2 baths,2.0,1.0,"[""Essentials"", ""Shampoo"", ""Smoke alarm"", ""Fire...",$145.00,30,1125,t,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN
4811,7556587,NaN,NaN,0.0,NaN,NaN,NaN,Harlem,40.82992,-73.94033,Private room in rental unit,Private room,2,1 bath,1.0,1.0,"[""Elevator"", ""Heating"", ""Smoke alarm"", ""Kitche...",$28.00,30,1125,t,0,0,0,0,1,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,f,0.01
19813,39112224,NaN,NaN,0.0,NaN,NaN,NaN,Midtown,40.75309,-73.97134,Entire condo,Entire home/apt,4,1 bath,1.0,2.0,"[""Gym"", ""Long term stays allowed"", ""TV with st...",$771.00,2,182,t,1,31,57,101,1,0,0,1.0,1.00,4.00,3.00,1.00,5.0,1.00,t,0.03
19956,39398026,NaN,NaN,0.0,NaN,NaN,NaN,Midtown,40.75318,-73.97345,Entire condo,Entire home/apt,4,1 bath,1.0,2.0,"[""Gym"", ""Long term stays allowed"", ""TV with st...",$771.00,2,182,t,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN
19961,39398031,NaN,NaN,0.0,NaN,NaN,NaN,Midtown,40.75346,-73.97340,Entire condo,Entire home/apt,4,1 bath,1.0,2.0,"[""Gym"", ""Long term stays allowed"", ""TV with st...",$771.00,2,182,t,2,32,58,102,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN
19963,39398034,NaN,NaN,0.0,NaN,NaN,NaN,Midtown,40.75380,-73.97191,Entire condo,Entire home/apt,2,1 bath,1.0,1.0,"[""Gym"", ""Long term stays allowed"", ""TV with st...",$643.00,2,182,t,1,31,57,101,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN
19969,39398035,NaN,NaN,0.0,NaN,NaN,NaN,Midtown,40.75221,-73.97160,Entire condo,Entire home/apt,2,1 bath,1.0,1.0,"[""Gym"", ""Long term stays allowed"", ""TV with st...",$643.00,2,182,t,1,31,57,101,3,0,0,3.0,3.33,4.67,4.67,3.33,5.0,3.33,t,0.09


I dropped the null values because each of the rows had more than 2 null values. 

In [30]:
df.dropna(subset = 'host_listings_count', inplace = True)

### host_identity_verified

I began by changing the column into 0 and 1 for false and true. 

In [31]:
df['host_identity_verified'] = df['host_identity_verified'].map({'f':0, 't':1})

The column had no null values so no rows will be dropped

In [32]:
df['host_identity_verified'].isnull().sum()

0

Most of the host for the airbnb are verified and inly 0.15 are not verified.

In [33]:
df['host_identity_verified'].value_counts(normalize = True)

1    0.844355
0    0.155645
Name: host_identity_verified, dtype: float64

I made sure to change the dtypes from object to int

In [34]:
df['host_identity_verified'] = df['host_identity_verified'].astype(int)

### Host_has_profile_pic

The column was changed from false and true to 0 and 1. 

In [35]:
df['host_has_profile_pic'] = df['host_has_profile_pic'].map({'f':0, 't':1})



### Property type

In [36]:
df['property_type'].value_counts()

Entire rental unit                    16840
Private room in rental unit           10466
Private room in home                   2132
Entire condo                           1625
Entire home                            1468
Private room in townhouse              1065
Entire loft                             746
Room in hotel                           666
Entire townhouse                        659
Entire serviced apartment               614
Private room in condo                   598
Room in boutique hotel                  465
Shared room in rental unit              430
Entire guest suite                      391
Private room in serviced apartment      298
Private room in loft                    273
Private room in guest suite             130
Private room in casa particular          93
Entire place                             76
Entire guesthouse                        65
Room in aparthotel                       63
Private room in bed and breakfast        63
Private room                    

Under the property types of rooms that there are in the airbnb, I decided to rename them all to Room rental unit to group them all up and make it more understable to read. 

In [37]:
df['property_type'].replace({
    'Room in hotel' : 'Room rental unit',
    'Room in boutique hotel' : 'Room rental unit',
    'Room in aparthotel': 'Room rental unit',
    'Room in serviced apartment' : 'Room rental unit',
    'Room in resort' : 'Room rental unit',
    'Room in bed and breakfast' : 'Room rental unit',
}, inplace = True)

There were also multiple rows with the word Entire at the start of them and so I decided to rename them all in the same group called Entire rental unit to make it understandable to read.

In [38]:
df['property_type'].replace({
    'Entire condo' : 'Entire rental unit',
    'Entire home': 'Entire rental unit',
    'Entire loft' : 'Entire rental unit',
    'Entire townhouse': 'Entire rental unit',
    'Entire serviced apartment': 'Entire rental unit',
    'Entire guest suite': 'Entire rental unit',
    'Entire place': 'Entire rental unit',
    'Entire guesthouse': 'Entire rental unit',
    'Entire vacation home': 'Entire rental unit',
    'Entire bungalow': 'Entire rental unit',
    'Entire cottage': 'Entire rental unit',
    'Entire home/apt': 'Entire rental unit',
    'Entire villa' : 'Entire rental unit',
    'Entire bed and breakfast' : 'Entire rental unit',
    }, inplace = True
)

The same is applied here as well as there are multiple rows that start with Private room. What was done was renaming all of these rows to a single name called Private room in rental unit to make it readible and understandable. 

In [39]:
df['property_type'].replace({
    'Private room in home': 'Private room in rental unit',
    'Private room in townhouse': 'Private room in rental unit',
    'Private room in condo': 'Private room in rental unit',
    'Private room in serviced apartment': 'Private room in rental unit',
    'Private room in loft' : 'Private room in rental unit',                    
    'Private room in guest suite' : 'Private room in rental unit',            
    'Private room in casa particular' : 'Private room in rental unit',
    'Private room in bed and breakfast' : 'Private room in rental unit',
    'Private room' : 'Private room in rental unit',
    'Private room in resort' : 'Private room in rental unit',
    'Private room in hostel': 'Private room in rental unit',
    'Private room in guesthouse': 'Private room in rental unit',
    'Private room in vacation home':'Private room in rental unit',
    'Private room in villa': 'Private room in rental unit',
    'Private room in bungalow': 'Private room in rental unit',
    'Private room in earthen home': 'Private room in rental unit',
    'Private room in tiny home': 'Private room in rental unit',
    'Private room in minsu' : 'Private room in rental unit',
    'Private room in houseboat': 'Private room in rental unit',
    'Private room in dorm': 'Private room in rental unit',
    'Private room in camper/rv': 'Private room in rental unit',
    'Private room in floor': 'Private room in rental unit',
    'Private room in tent': 'Private room in rental unit',
    'Private room in religious building': 'Private room in rental unit',     
    'Private room in in-law' : 'Private room in rental unit',                    
    'Private room in farm stay' : 'Private room in rental unit', 
    'Private room in lighthouse' : 'Private room in rental unit',
    'Private room in train' : 'Private room in rental unit',
    'Private room in cottage' : 'Private room in rental unit',               
    'Private room in barn' : 'Private room in rental unit',
}, inplace = True)

Multiple rows shared a similar start with the name Shared room and so they were all renamed to Shared room in rental unit to make it easier to read and understand the rows.

In [40]:
df['property_type'].replace({
    'Shared room in home' : 'Shared room in rental unit',
    'Shared room in condo' : 'Shared room in rental unit',
    'Shared room in townhouse': 'Shared room in rental unit',
    'Shared room in hostel': 'Shared room in rental unit',
    'Shared room in bed and breakfast': 'Shared room in rental unit',
    'Shared room in serviced apartment': 'Shared room in rental unit',
    'Shared room in vacation home': 'Shared room in rental unit',
    'Shared room in guest suite' : 'Shared room in rental unit',
    'Shared room in guesthouse' : 'Shared room in rental unit',
    'Shared room': 'Shared room in rental unit',
    'Shared room in floor': 'Shared room in rental unit', 
    'Shared room in bungalow' : 'Shared room in rental unit',
    'Shared room in casa particular' : 'Shared room in rental unit',
    'Shared room in loft': 'Shared room in rental unit',
}, inplace = True)

These particular property types of rental locations I thought were miscellaneous or additional property types that were rentable, and so I decided to group them all up by renaming them as Other.

In [41]:
df['property_type'].replace({
    'Casa particular' : 'Other',                
    'Tiny home' : 'Other',                     
    'Boat' : 'Other',                               
    'Houseboat' : 'Other',                           
    'Floor' : 'Other',                               
    'Camper/RV' : 'Other',                                    
    'Barn' : 'Other',                                
    'Tower' : 'Other',                              
    'Lighthouse' : 'Other',                          
    'Cave': 'Other',                                
    'Castle': 'Other',                              
    'Tent': 'Other', 
}, inplace = True)

### Room_type

The room_type column show the different types of rooms there are in airbnb rentals.

In [42]:
df['room_type'].value_counts(normalize = True)

Entire home/apt    0.571177
Private room       0.410482
Shared room        0.013994
Hotel room         0.004347
Name: room_type, dtype: float64

### Accomodates

The accomodates column shows the percentages of airbnbs that accomodates people. 

In [43]:
df['accommodates'].value_counts(normalize = True)

2     0.445053
4     0.161072
1     0.153435
3     0.100472
6     0.052158
5     0.043340
8     0.016105
7     0.010979
10    0.005552
16    0.003945
12    0.003015
9     0.002186
11    0.000854
15    0.000829
14    0.000603
13    0.000402
Name: accommodates, dtype: float64

### Bathroom_text

First, I will begin by renaming this bathroom to keep it simpler. 

In [44]:
df.rename(columns = {'bathrooms_text':'bathrooms'}, inplace = True)

I am finding out how many null values that there are under the bathrooms column.

In [45]:
df['bathrooms'].isnull().sum()

55

In [46]:
df[df['bathrooms'].isna()]

,id,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month
2,5121,1.00,1.00,1.0,2.0,1,1,Bedford-Stuyvesant,40.685350,-73.955120,Private room in rental unit,Private room,2,NaN,1.0,1.0,"[""Heating"", ""Kitchen"", ""Air conditioning"", ""Wi...",$60.00,30,730,t,5,30,60,335,50,0,0,4.52,4.22,4.09,4.91,4.91,4.47,4.52,f,0.31
146,127387,NaN,NaN,0.0,1.0,1,1,Gowanus,40.668950,-73.991360,Entire rental unit,Entire home/apt,4,NaN,2.0,2.0,"[""Heating"", ""Shampoo"", ""Smoke alarm"", ""TV"", ""F...",$260.00,30,60,t,0,0,0,221,3,0,0,5.00,5.00,5.00,5.00,5.00,4.67,5.00,f,0.02
190,68253,NaN,NaN,0.0,1.0,1,1,Astoria,40.763360,-73.920990,Private room in rental unit,Private room,1,NaN,1.0,NaN,"[""Long term stays allowed""]",$55.00,30,730,f,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN
203,39572,0.67,0.00,0.0,2.0,1,1,Hell's Kitchen,40.762030,-73.988690,Private room in rental unit,Private room,2,NaN,1.0,1.0,"[""Hair dryer"", ""Essentials"", ""Iron"", ""Dishes a...",$139.00,30,1125,t,0,0,0,6,44,0,0,4.86,4.93,4.83,4.98,5.00,4.98,4.86,f,0.30
224,154934,0.40,0.33,0.0,2.0,1,1,Harlem,40.824100,-73.944150,Private room in rental unit,Private room,1,NaN,1.0,1.0,"[""Elevator"", ""Heating"", ""Smoke alarm"", ""Kitche...",$75.00,30,90,t,6,36,66,341,38,0,0,4.42,4.71,4.03,4.79,4.79,4.15,4.42,f,0.30
237,74438,NaN,NaN,0.0,3.0,1,0,Hell's Kitchen,40.755230,-73.997920,Entire rental unit,Entire home/apt,5,NaN,1.0,1.0,"[""Cable TV"", ""Elevator"", ""Heating"", ""Shampoo"",...",$299.00,30,365,t,0,0,0,218,52,0,0,4.84,4.90,4.54,4.79,4.92,4.79,4.71,f,0.37
337,200645,NaN,NaN,0.0,1.0,1,0,Upper East Side,40.767390,-73.955700,Shared room in rental unit,Shared room,1,NaN,1.0,1.0,"[""Cable TV"", ""Elevator"", ""Kitchen"", ""Dryer"", ""...",$90.00,30,365,f,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN
348,254409,NaN,NaN,0.0,1.0,1,1,Clinton Hill,40.681360,-73.965400,Entire rental unit,Entire home/apt,3,NaN,1.0,1.0,"[""Essentials"", ""Elevator"", ""Heating"", ""Kitchen...",$110.00,1,1125,t,0,0,0,0,2,0,0,4.50,4.50,4.00,5.00,4.50,5.00,4.50,f,0.02
380,310325,0.40,0.33,0.0,2.0,1,1,Harlem,40.823590,-73.946010,Private room in rental unit,Private room,1,NaN,1.0,1.0,"[""Elevator"", ""Heating"", ""Kitchen"", ""Wifi"", ""Lo...",$75.00,30,60,t,1,1,28,303,34,0,0,4.17,3.97,3.71,4.74,4.65,4.19,4.03,f,0.28
589,499249,NaN,NaN,0.0,1.0,1,1,Williamsburg,40.705280,-73.957560,Private room in rental unit,Private room,1,NaN,1.0,NaN,"[""Long term stays allowed""]",$190.00,30,218,f,0,0,0,0,1,0,0,0.00,5.00,4.00,5.00,5.00,5.00,5.00,f,0.01


I will drop the bathrooms NaN due to the fact that each property type may have a number of bathrooms and not having an accurate reading on could hinder the model. At the same time it was 55 null values which is less than 5% of the data, so we aren't losing too much

In [47]:
df.dropna(subset = 'bathrooms', inplace = True)

After looking at the value_counts of the bathrooms column, I wanted to see the top 3 types of bathrooms that were most frequent in airbnb rentals. The top three were 1 bath, 1 shared bath, and 1 private bath at 21061, 9169, and 2817 respectively. 

In [48]:
df['bathrooms'].value_counts()

1 bath               21061
1 shared bath         9169
1 private bath        2817
2 baths               2599
1.5 baths             1112
2 shared baths        1044
1.5 shared baths       698
2.5 baths              383
3 baths                246
0 shared baths          98
3 shared baths          92
3.5 baths               88
2.5 shared baths        79
4 baths                 65
0 baths                 51
Half-bath               44
4.5 baths               26
Shared half-bath        19
Private half-bath       12
4 shared baths          12
5.5 baths                9
6 baths                  7
5 baths                  6
4.5 shared baths         2
6.5 baths                2
6 shared baths           2
3.5 shared baths         1
7.5 baths                1
7 baths                  1
9.5 baths                1
Name: bathrooms, dtype: int64

### Amenities

In [49]:
df.head()

,id,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month
0,2539,1.00,0.90,0.0,9.0,1,1,Kensington,40.64529,-73.97238,Private room in rental unit,Private room,2,1 shared bath,1.0,1.0,"[""First aid kit"", ""Microwave"", ""Stove"", ""Coffe...",$299.00,30,730,t,21,51,81,356,9,0,0,4.89,4.88,5.00,5.00,5.0,4.75,4.88,f,0.11
1,2595,0.75,0.23,0.0,6.0,1,1,Midtown,40.75356,-73.98559,Entire rental unit,Entire home/apt,1,1 bath,NaN,1.0,"[""Stove"", ""Coffee maker"", ""Long term stays all...",$175.00,30,1125,t,0,0,5,280,49,1,0,4.68,4.73,4.63,4.77,4.8,4.81,4.40,f,0.31
3,45910,1.00,0.19,0.0,7.0,1,1,Ridgewood,40.70309,-73.89963,Entire rental unit,Entire home/apt,16,2.5 baths,5.0,10.0,"[""Hair dryer"", ""Essentials"", ""Carbon monoxide ...",$425.00,30,730,t,30,60,90,365,13,0,0,4.42,4.64,4.36,4.82,5.0,4.82,4.55,f,0.10
4,5136,NaN,0.33,0.0,1.0,1,1,Sunset Park,40.66265,-73.99454,Entire rental unit,Entire home/apt,4,1.5 baths,2.0,2.0,"[""Hair dryer"", ""Cable TV"", ""Refrigerator"", ""BB...",$275.00,21,1125,t,0,0,0,179,3,1,1,5.00,5.00,5.00,5.00,5.0,4.67,5.00,f,0.03
5,45935,NaN,NaN,0.0,1.0,1,0,Mott Haven,40.80635,-73.92201,Private room in rental unit,Private room,1,1 shared bath,1.0,1.0,"[""First aid kit"", ""Essentials"", ""Breakfast"", ""...",$60.00,30,40,t,23,53,83,83,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN


Amenities contains a list and so I separated them the most important amenities that I thought were important as well as the most common ones that I saw were categorized under certain new columns. The columns that I made and categorize certain amenities in were tv, heating, gym, internet, parking, kitchen_supply, bathroom_supply, laundry, entrance, stay, and secure.

In [50]:
df['tv'] = df['amenities'].str.contains('TV| Bluetooth sound system|Xbox One|Amazon Prime Video|PS4')
df['heating'] = df['amenities'].str.contains('Heating')
df['gym'] = df['amenities'].str.contains('Exercise equipment|Gym|gym')
df['internet'] = df['amenities'].str.contains('Internet|Wifi|Fast Wifi')
df['parking'] = df['amenities'].str.contains('Free street parking|Free parking on premises|Paid parking off premises')
df['kitchen_supply'] = df['amenities'].str.contains('Microwave|Coffee maker|Oven|Kitchen|Stove|Dishes and silverware|Dishwasher|Refrigerator|Cooking basics')
df['bathroom_supply'] = df['amenities'].str.contains('Hair dryer|Shampoo|Bathtub')
df['laundry'] = df['amenities'].str.contains('Iron|Washer|Dryer')
df['entrance'] = df['amenities'].str.contains('Private entrance')
df['stay'] = df['amenities'].str.contains('Long term stays allowed')
df['secure'] = df['amenities'].str.contains('Security cameras on property|Lock on bedroom door|Keypad|Smart lock')


When I made these new columns for each amenities category, there had dtypes as objects. So I decided to change them all into int.

In [51]:
df['tv'] = df['tv'].astype(int)
df['heating'] = df['heating'].astype(int)
df['gym'] = df['gym'].astype(int)
df['internet'] = df['internet'].astype(int)
df['parking'] = df['parking'].astype(int)
df['kitchen_supply'] = df['kitchen_supply'].astype(int)
df['bathroom_supply'] = df['bathroom_supply'].astype(int)
df['laundry'] = df['laundry'].astype(int)
df['entrance'] = df['entrance'].astype(int)
df['stay'] = df['stay'].astype(int)
df['secure'] = df['secure'].astype(int)

I will not need the amenities column since I made new columns that categorizes the amenities into a group so the amenities column was droppped.

In [52]:
df.drop(columns = 'amenities', inplace = True)

Amenities Columns:
- TV Column: TV, Bluetooth sound system, Xbox One, Amazon Prime Video, PS4
- Heating Column : Heating
- Gym Column : Exercise Equipment, GYM, and gym
- Internet Column: Internet, Wifi, Fast Wifi
- Parking Column : Free Street Parking, Free Parking on Premis, Paid Parking off Premise
- Kitchen Supply Column : Microwave, Coffee Maker, Oven, Kitchen, Stove, Dishes and Silverware, Dishwasher, Refrigerator, Cooking basics
- Bathroom Supply COlumn : Hair Dryer, Shampoo, and Bathtub
- Laundry Column : Iron, Washer, Dryer
- Entrance Column : Private Entrance
- Stay Column : Long Term Stay Allowed
- Secure Column : Security Cameras on Property. Lock on bedroom doors, Keypad, and Smart Lock

### Price

In [53]:
df['price']

0        $299.00
1        $175.00
3        $425.00
4        $275.00
5         $60.00
          ...   
39876    $157.00
39877    $108.00
39878    $140.00
39879    $400.00
39880     $40.00
Name: price, Length: 39747, dtype: object

The price column had values containing $ and , and so they were not under the correct dtypes and so I decided to replace these characters so that I could remove them and convert it to their appropriate dtype as a float. 

In [54]:
df['price'] = df['price'].str.replace('$', '')
df['price'] = df['price'].str.replace(',', '')
df['price'] = df['price'].astype(float)

/var/folders/yw/mzjjwqwd12g6lm2qkdn097dw0000gn/T/ipykernel_10335/3445222675.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['price'] = df['price'].str.replace('$', '')


I noticed some outliers in the prices of airbnb locations and so I decided to clean up the column to keep any data from being skewed.

In [56]:
df = df[df['price']<= 1250]

### Minimum Nights

There were some outliers when looking at the boxplot that can be seen from the boxplot in EDA/Data Cleaning. They will be cleaned to keep the data from being too skewed. 

In [59]:
df = df[df['minimum_nights']<= 365]

### Availability

has_availability will be changed into 0 and 1 for false and true.

In [60]:

df['has_availability'] = df['has_availability'].map({'f':0, 't':1}).astype(int)

I want to look into two categories in which I look at the next 30 days and base the pricing off of that and the possibility of generalizing so I will look at the 365 day availability as well. 

In [61]:
df.drop(columns = ['availability_60', 'availability_90'], inplace = True)

I did a value_counts to see the top three availability_30s which were 0, 1, and 6 with values of 21209, 1523, and 1286 respectively.

In [62]:
df['availability_30'].value_counts()

0     21209
1      1523
6      1286
2      1252
3      1160
30     1101
4      1024
7       983
5       969
8       614
9       545
12      534
10      527
11      497
13      472
15      472
14      449
23      436
16      436
17      397
29      391
19      353
20      339
18      334
22      320
28      316
24      309
26      295
21      295
27      294
25      250
Name: availability_30, dtype: int64

I also did a value_counts for availability_365 to find the top 3 availability_365s which were 0, 365, and 341 which had values of 14206, 752, and 389 respectively.

In [63]:
df['availability_365'].value_counts()

0      14206
365      752
341      389
249      331
342      252
       ...  
205       16
206       16
227       14
203       11
194       11
Name: availability_365, Length: 366, dtype: int64

### Instant_Bookability

The instant_bookable column had values that could be either false or true and so I decided to change false to 0 and true to 1.

In [64]:
df['instant_bookable'] = df['instant_bookable'].map({'f':0, 't':1})

In [65]:
df.to_csv('./data/df_cleaning_01.csv')

The categorical columns were dummified

In [66]:
categorical_column = df.select_dtypes(include=['object']).columns
df_dum = pd.get_dummies(df, columns = categorical_column, drop_first = True, dtype = int)

In [67]:
categorical_column

Index(['neighbourhood_cleansed', 'property_type', 'room_type', 'bathrooms'], dtype='object')

Looked at the number of columns after dummifying the columns

In [68]:
len(df_dum.columns)

318

Now that we looked at the columns that had null values, we will do an iterative imputer with linear regression with the missing data from columns that had a large number of missing data. 

Train/Test Split was performed in which iterative imputer will be used to fill the null values

In [69]:
X = df_dum.drop(columns = 'price')
y = df_dum['price']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

An iterative imputer was used with linear regression 

In [70]:
it = IterativeImputer(estimator = LinearRegression())
X_train_imputed = it.fit_transform(X_train)
X_test_imputed = it.transform(X_test)

A dataframe was made for the train and test for when modeling was used. 

In [71]:
df_train = pd.DataFrame(X_train_imputed, columns = X_train.columns, index = y_train.index)

df_test = pd.DataFrame(X_test_imputed, columns = X_test.columns, index = y_test.index)

In [72]:
imputed_train = df_train.merge(right = df['price'],
                                how = 'left',
                                left_index = True,
                                right_index = True).sort_index()
imputed_test = df_test.merge(right = df['price'], 
                             how = 'left',
                            left_index = True,
                            right_index = True).sort_index()

In [73]:
imputed_train.to_csv('./data/imputed_train.csv', index = True)
imputed_test.to_csv('./data/imputed_test.csv', index = True)

Columns Included in Dataframe:
-  Id
- host_response_rate
- host_acceptance rate
- host_is_superhost
- host_listings_count
- host_has_profile_pic
- host_identity_verified
- latitidue
- longitude
- accomodates
- bedrooms
- beds
- Minimum_nights
- maximum_nights
- has_availability
- availability_30
- availability_365
- number_of_reviews
- number_of_reviews_ltm
- number_of_reviews_30d
- review_scores_rating
- review_Scores_accuracy
- review_scores_cleanliness
- review_scores_checkin
- review_Scores_communication
- review_scores_location
- review_scores_value
- instant_bookable
- review_per_month
- tv
- heating
- gym
- internet
- parking
- kicthen_supply
- bathroom_supply
- laundry
- entrance
- stay
- secure